# Where to Open a Japanese Restaurant in San Diego?

## A recent study by Zagat (https://qz.com/657013/what-americans-are-willing-to-pay-for-ethnic-restaurant-food-reveals-some-pretty-deep-prejudices/) demonstrated that Japanese food is the second most expensive cultural food that American's will spend money on.  Due to this, when deciding where to open a new Japanese restaurant, restauranteers should look to zip codes that are above the mean household income of the US, as well as a low number of existing Japanese restaurants

### I set out to determine which zip codes in San Diego offer a low number of competing locations within zip codes above the mean household income for the US

#### Import Necessary Libraries

In [62]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


#### Credentials for using FourSquare

In [63]:
CLIENT_ID = 'BZAOTE3PBPFSEZ4MBS44FD124XOQP1Z2R4DGJENDAYT1CTB0' 
CLIENT_SECRET = '2PAQDTDAMNIPMDD0PPOBN5JUENEJTHCAPSGGYOHUS0YRJV0I'
CODE = 'CULEFRQGLJRYLKIRI0ESVLS3QZPYFJ2QQVEQWCSN5P3STWX1#_=_'
ACCESS_TOKEN = 'HAVG3E4KDUFCRZUBUMTQQVFSZFWXVBD1I1MIF41K0U1I3J3K'
VERSION = '20201229'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('ACCESS_TOKEN:' + ACCESS_TOKEN)

Your credentails:
CLIENT_ID: BZAOTE3PBPFSEZ4MBS44FD124XOQP1Z2R4DGJENDAYT1CTB0
CLIENT_SECRET:2PAQDTDAMNIPMDD0PPOBN5JUENEJTHCAPSGGYOHUS0YRJV0I
ACCESS_TOKEN:HAVG3E4KDUFCRZUBUMTQQVFSZFWXVBD1I1MIF41K0U1I3J3K


#### I will start by looking at Japanese restaurants using downtown San Diego Courthouse as the center to begin the map search.  Will start with a 50 mile radius as San Diego county is a top 10 county for population in the US

In order to use FourSquare I need to use an agent which I will call: "foursquare_agent"

In [64]:
address = '1100 Union Street, San Diego, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

32.716855 -117.165704


In [7]:
search_query = 'Japanese'
radius = 80500
print(search_query + ' .... OK!')

Japanese .... OK!


In [17]:
url = 'https://api.foursquare.com/v2/venues/search?client_id=BZAOTE3PBPFSEZ4MBS44FD124XOQP1Z2R4DGJENDAYT1CTB0&client_secret=2PAQDTDAMNIPMDD0PPOBN5JUENEJTHCAPSGGYOHUS0YRJV0I&ll=32.716855,-117.165704&oauth_token=HAVG3E4KDUFCRZUBUMTQQVFSZFWXVBD1I1MIF41K0U1I3J3K&v=20201229&query=Japanese&radius=80500&limit=30'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=BZAOTE3PBPFSEZ4MBS44FD124XOQP1Z2R4DGJENDAYT1CTB0&client_secret=2PAQDTDAMNIPMDD0PPOBN5JUENEJTHCAPSGGYOHUS0YRJV0I&ll=32.716855,-117.165704&oauth_token=HAVG3E4KDUFCRZUBUMTQQVFSZFWXVBD1I1MIF41K0U1I3J3K&v=20201229&query=Japanese&radius=80500&limit=30'

#### send the GET requests and examine the results

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff351042b881e51cdde7b12'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '483d01cef964a52019501fe3',
    'name': 'Japanese Friendship Garden',
    'location': {'address': '2215 Pan American Rd E',
     'crossStreet': 'in Balboa Park',
     'lat': 32.73005388762262,
     'lng': -117.14998483657837,
     'labeledLatLngs': [{'label': 'display',
       'lat': 32.73005388762262,
       'lng': -117.14998483657837},
      {'label': 'entrance', 'lat': 32.730109, 'lng': -117.15003}],
     'distance': 2079,
     'postalCode': '92101',
     'cc': 'US',
     'city': 'San Diego',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['2215 Pan American Rd E (in Balboa Park)',
      'San Diego, CA 92101']},
    'categories': [{'id': '4bf58dd8d48988d15a941735',
      'name': 'Garden',
      'pluralName': 'Gardens',
      'shortName': 'Garden',
      'icon': {'prefix': 'https:/

#### Get the relevant parts of the JSON file and convert to a pandas dataframe

In [20]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,venuePage.id,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,483d01cef964a52019501fe3,Japanese Friendship Garden,"[{'id': '4bf58dd8d48988d15a941735', 'name': 'G...",v-1609781508,False,2215 Pan American Rd E,in Balboa Park,32.730054,-117.149985,"[{'label': 'display', 'lat': 32.73005388762262...",...,United States,"[2215 Pan American Rd E (in Balboa Park), San ...",45465452,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4dcc330dd22d480bd645169b,Katsu Japanese Cuisine,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1609781508,False,4th Ave,NaN,32.727264,-117.161042,"[{'label': 'display', 'lat': 32.7272640624101,...",...,United States,"[4th Ave, San Diego, CA 92101]",NaN,Park West,NaN,NaN,NaN,NaN,NaN,NaN
2,5912fef1a6fe4d28dcf09b80,Japanese Auto Sales,"[{'id': '4eb1c1623b7b52c0e1adc2ec', 'name': 'A...",v-1609781508,False,4825 A El Cajon Blvd,NaN,32.717219,-117.159705,"[{'label': 'display', 'lat': 32.7172188597113,...",...,United States,"[4825 A El Cajon Blvd, San Diego, CA 92115]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50651697e4b0d3feb102b8f7,Sora Italian Japanese Influenced Cuisine,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1609781508,False,655 W Broadway,Kettner,32.715151,-117.168581,"[{'label': 'display', 'lat': 32.715151, 'lng':...",...,United States,"[655 W Broadway (Kettner), San Diego, CA 92101]",NaN,Central San Diego,NaN,NaN,NaN,NaN,NaN,NaN
4,5f93a630aecea61d0fa246ce,Gyu-Kaku Japanese BBQ,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1609781508,False,801 5th Ave,F St.,32.713734,-117.159976,"[{'label': 'display', 'lat': 32.713734, 'lng':...",...,United States,"[801 5th Ave (F St.), San Diego, CA 92101]",NaN,NaN,2339200,https://www.grubhub.com/restaurant/gyu-kaku-80...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png


In [22]:

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Japanese Friendship Garden,Garden,2215 Pan American Rd E,in Balboa Park,32.730054,-117.149985,"[{'label': 'display', 'lat': 32.73005388762262...",2079,92101,US,San Diego,CA,United States,"[2215 Pan American Rd E (in Balboa Park), San ...",NaN,483d01cef964a52019501fe3
1,Katsu Japanese Cuisine,Japanese Restaurant,4th Ave,NaN,32.727264,-117.161042,"[{'label': 'display', 'lat': 32.7272640624101,...",1238,92101,US,San Diego,CA,United States,"[4th Ave, San Diego, CA 92101]",Park West,4dcc330dd22d480bd645169b
2,Japanese Auto Sales,Auto Dealership,4825 A El Cajon Blvd,NaN,32.717219,-117.159705,"[{'label': 'display', 'lat': 32.7172188597113,...",563,92115,US,San Diego,CA,United States,"[4825 A El Cajon Blvd, San Diego, CA 92115]",NaN,5912fef1a6fe4d28dcf09b80
3,Sora Italian Japanese Influenced Cuisine,Japanese Restaurant,655 W Broadway,Kettner,32.715151,-117.168581,"[{'label': 'display', 'lat': 32.715151, 'lng':...",329,92101,US,San Diego,CA,United States,"[655 W Broadway (Kettner), San Diego, CA 92101]",Central San Diego,50651697e4b0d3feb102b8f7
4,Gyu-Kaku Japanese BBQ,Japanese Restaurant,801 5th Ave,F St.,32.713734,-117.159976,"[{'label': 'display', 'lat': 32.713734, 'lng':...",639,92101,US,San Diego,CA,United States,"[801 5th Ave (F St.), San Diego, CA 92101]",NaN,5f93a630aecea61d0fa246ce
5,Koi Japanese Restaurant,Japanese Restaurant,744 Market St,8th Ave,32.711650,-117.157730,"[{'label': 'display', 'lat': 32.71165, 'lng': ...",945,92101,US,San Diego,CA,United States,"[744 Market St (8th Ave), San Diego, CA 92101]",NaN,48701183f964a52004511fe3
6,The Tea Pavillion at the Japanese Friendship G...,Tea Room,2215 Pan American Rd E,in Balboa Park,32.730113,-117.149981,"[{'label': 'display', 'lat': 32.73011346770401...",2084,92101,US,San Diego,CA,United States,"[2215 Pan American Rd E (in Balboa Park), San ...",Balboa Park,4ba00ebef964a5205e5637e3
7,Yoshino Japanese Restaurant,Japanese Restaurant,1790 W Washington St,at India St,32.743662,-117.181338,"[{'label': 'display', 'lat': 32.74366238711516...",3323,92103,US,San Diego,CA,United States,"[1790 W Washington St (at India St), San Diego...",Mission Hills,44533d88f964a520ab321fe3
8,san diego japanese christian church,Church,NaN,NaN,32.715037,-117.146453,"[{'label': 'display', 'lat': 32.7150365055895,...",1814,NaN,US,San Diego,CA,United States,"[San Diego, CA]",NaN,4ea856eccc21adeb9944927b
9,Oriental Treasure Box: Japanese Antiques & Arts,Antique Shop,2310 Kettner Blvd,NaN,32.728222,-117.171661,"[{'label': 'display', 'lat': 32.72822189331055...",1382,92101,US,San Diego,CA,United States,"[2310 Kettner Blvd, San Diego, CA 92101]",NaN,4ec813d09adf9c7bf3c4e6b0


#### There are a number of results for 'Japanese' that are not restaurants.  Need to filter these results to only those which are restaurants

In [45]:
dataframe_restaurant = dataframe_filtered.set_index('categories').filter(like='Japanese Restaurant' or 'Sushi Restaurant' or 'Ramen Restaurant' or 'Asian Restaurant', axis=0)

In [46]:
dataframe_restaurant

,name,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
categories,,,,,,,,,,,,,,,
Japanese Restaurant,Katsu Japanese Cuisine,4th Ave,NaN,32.727264,-117.161042,"[{'label': 'display', 'lat': 32.7272640624101,...",1238,92101,US,San Diego,CA,United States,"[4th Ave, San Diego, CA 92101]",Park West,4dcc330dd22d480bd645169b
Japanese Restaurant,Sora Italian Japanese Influenced Cuisine,655 W Broadway,Kettner,32.715151,-117.168581,"[{'label': 'display', 'lat': 32.715151, 'lng':...",329,92101,US,San Diego,CA,United States,"[655 W Broadway (Kettner), San Diego, CA 92101]",Central San Diego,50651697e4b0d3feb102b8f7
Japanese Restaurant,Gyu-Kaku Japanese BBQ,801 5th Ave,F St.,32.713734,-117.159976,"[{'label': 'display', 'lat': 32.713734, 'lng':...",639,92101,US,San Diego,CA,United States,"[801 5th Ave (F St.), San Diego, CA 92101]",NaN,5f93a630aecea61d0fa246ce
Japanese Restaurant,Koi Japanese Restaurant,744 Market St,8th Ave,32.711650,-117.157730,"[{'label': 'display', 'lat': 32.71165, 'lng': ...",945,92101,US,San Diego,CA,United States,"[744 Market St (8th Ave), San Diego, CA 92101]",NaN,48701183f964a52004511fe3
Japanese Restaurant,Yoshino Japanese Restaurant,1790 W Washington St,at India St,32.743662,-117.181338,"[{'label': 'display', 'lat': 32.74366238711516...",3323,92103,US,San Diego,CA,United States,"[1790 W Washington St (at India St), San Diego...",Mission Hills,44533d88f964a520ab321fe3
Japanese Restaurant,SanSai Japanese Grill,7710 Hazard Center Dr,at Frazee Rd,32.771254,-117.155877,"[{'label': 'display', 'lat': 32.77125422339901...",6125,92108,US,San Diego,CA,United States,"[7710 Hazard Center Dr (at Frazee Rd), San Die...",NaN,4ad3e184f964a520c2e620e3
Japanese Restaurant,Furasshu Japanese Cuisine,Blvd. Cuauhtemoc 10237 Zona Río,A Un Lado De Total Fitness,32.523962,-117.019051,"[{'label': 'display', 'lat': 32.52396194685493...",25497,22010,MX,Tijuana,Baja California,México,[Blvd. Cuauhtemoc 10237 Zona Río (A Un Lado De...,NaN,5029544fe4b00bbc7b40d9f8
Japanese Restaurant,Hanaya Japanese Restaurant,6950 Federal Blvd,NaN,32.741890,-117.048866,"[{'label': 'display', 'lat': 32.74188995361328...",11290,91945,US,Lemon Grove,CA,United States,"[6950 Federal Blvd, Lemon Grove, CA 91945]",NaN,5d731c715c43dd0007a2cd5d
Japanese Restaurant,Fuji Japanese Steakhouse & Sushi,911 Camino del Rio S,NaN,32.763673,-117.155412,"[{'label': 'display', 'lat': 32.76367258150530...",5300,92108,US,San Diego,CA,United States,"[911 Camino del Rio S, San Diego, CA 92108]",NaN,4ac82360f964a520c9bb20e3


#### Vizualize the location of the restaurants

In [57]:
dataframe_restaurant.postalCode.unique()

array(['92101', '92103', '92108', '22010', '91945', '92111'], dtype=object)

In [116]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around the Downtown Courthouse

# add a red circle marker to represen downtown San Diego
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Downtown',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Japanese restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### I want to identify different clusters based on Japanese restaurant densities, so I will run clustering on the data to identify the location of areas with fewer Japanese restaurants.  

### Clustering - Add an integer column

In [72]:
# Need to update the df to include integer data for the categorical Borough in order to use clustering
dataframe_restaurant['Index']=dataframe_restaurant['postalCode'].replace(to_replace=['92101','92103','92108','92111','91945','22010'],value=[1,2,3,4,5,6],inplace=False)
dataframe_restaurant.head()

,name,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,Index
categories,,,,,,,,,,,,,,,,
Japanese Restaurant,Katsu Japanese Cuisine,4th Ave,NaN,32.727264,-117.161042,"[{'label': 'display', 'lat': 32.7272640624101,...",1238,92101,US,San Diego,CA,United States,"[4th Ave, San Diego, CA 92101]",Park West,4dcc330dd22d480bd645169b,1
Japanese Restaurant,Sora Italian Japanese Influenced Cuisine,655 W Broadway,Kettner,32.715151,-117.168581,"[{'label': 'display', 'lat': 32.715151, 'lng':...",329,92101,US,San Diego,CA,United States,"[655 W Broadway (Kettner), San Diego, CA 92101]",Central San Diego,50651697e4b0d3feb102b8f7,1
Japanese Restaurant,Gyu-Kaku Japanese BBQ,801 5th Ave,F St.,32.713734,-117.159976,"[{'label': 'display', 'lat': 32.713734, 'lng':...",639,92101,US,San Diego,CA,United States,"[801 5th Ave (F St.), San Diego, CA 92101]",NaN,5f93a630aecea61d0fa246ce,1
Japanese Restaurant,Koi Japanese Restaurant,744 Market St,8th Ave,32.711650,-117.157730,"[{'label': 'display', 'lat': 32.71165, 'lng': ...",945,92101,US,San Diego,CA,United States,"[744 Market St (8th Ave), San Diego, CA 92101]",NaN,48701183f964a52004511fe3,1
Japanese Restaurant,Yoshino Japanese Restaurant,1790 W Washington St,at India St,32.743662,-117.181338,"[{'label': 'display', 'lat': 32.74366238711516...",3323,92103,US,San Diego,CA,United States,"[1790 W Washington St (at India St), San Diego...",Mission Hills,44533d88f964a520ab321fe3,2


In [60]:
#Import Libraries for the map
!pip install geopy
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np 

In [73]:
#coordinates of San Diego to begin the clustering

address = 'San Diego'
geolocator = Nominatim(user_agent="SD_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of San Diego are {latitude}, {longitude}.')


The geograpical coordinate of San Diego are 32.7174202, -117.1627728.


In [74]:
#set the cluster number as the Index number created above
kclusters=len(dataframe_restaurant.Index.unique())

#create map of Toronto
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters of boroughs in Toronto
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add the zip code clusters to the map of San Diego
markers_colors = []
for lat, lon, cluster in zip(dataframe_restaurant['lat'], dataframe_restaurant['lng'], dataframe_restaurant['Index']):
    label = folium.Popup(str(dataframe_restaurant['postalCode']) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [75]:
map_clusters

### Based on clustering, there are 5 distinct areas, with the downtown area more densly populated.  New Japanese restaurants should be excluded from area code = 92101

## Will install data for median, mean household incomes in the US to compare to the Japanese restaurant distribution

In [82]:
#Install household income by zipcode
!wget -q -O 'income' https://www.psc.isr.umich.edu/dis/census/Features/tract2zip/MedianZIP-3.xlsx
print('Data downloaded!')

Data downloaded!


In [83]:
# read the excel file and convert into a dataframe object 
df_income = pd.DataFrame(pd.read_excel("income"))

In [84]:
df_income.head()

,Zip,Median,Mean,Pop
0,1001,56662.5735,66687.8,16445
1,1002,49853.4177,75062.6,28069
2,1003,28462.0000,35121,8491
3,1005,75423.0000,82442,4798
4,1007,79076.3540,85802,12962


### Add the Median income to the restaurants dataframe

In [85]:
dataframe_restaurant = dataframe_restaurant.rename(columns = {"postalCode":"Zip"})
dataframe_restaurant.head()

,name,address,crossStreet,lat,lng,labeledLatLngs,distance,Zip,cc,city,state,country,formattedAddress,neighborhood,id,Index
categories,,,,,,,,,,,,,,,,
Japanese Restaurant,Katsu Japanese Cuisine,4th Ave,NaN,32.727264,-117.161042,"[{'label': 'display', 'lat': 32.7272640624101,...",1238,92101,US,San Diego,CA,United States,"[4th Ave, San Diego, CA 92101]",Park West,4dcc330dd22d480bd645169b,1
Japanese Restaurant,Sora Italian Japanese Influenced Cuisine,655 W Broadway,Kettner,32.715151,-117.168581,"[{'label': 'display', 'lat': 32.715151, 'lng':...",329,92101,US,San Diego,CA,United States,"[655 W Broadway (Kettner), San Diego, CA 92101]",Central San Diego,50651697e4b0d3feb102b8f7,1
Japanese Restaurant,Gyu-Kaku Japanese BBQ,801 5th Ave,F St.,32.713734,-117.159976,"[{'label': 'display', 'lat': 32.713734, 'lng':...",639,92101,US,San Diego,CA,United States,"[801 5th Ave (F St.), San Diego, CA 92101]",NaN,5f93a630aecea61d0fa246ce,1
Japanese Restaurant,Koi Japanese Restaurant,744 Market St,8th Ave,32.711650,-117.157730,"[{'label': 'display', 'lat': 32.71165, 'lng': ...",945,92101,US,San Diego,CA,United States,"[744 Market St (8th Ave), San Diego, CA 92101]",NaN,48701183f964a52004511fe3,1
Japanese Restaurant,Yoshino Japanese Restaurant,1790 W Washington St,at India St,32.743662,-117.181338,"[{'label': 'display', 'lat': 32.74366238711516...",3323,92103,US,San Diego,CA,United States,"[1790 W Washington St (at India St), San Diego...",Mission Hills,44533d88f964a520ab321fe3,2


In [89]:
dataframe_restaurant.dtypes

name                 object
address              object
crossStreet          object
lat                 float64
lng                 float64
labeledLatLngs       object
distance              int64
Zip                  object
cc                   object
city                 object
state                object
country              object
formattedAddress     object
neighborhood         object
id                   object
Index                 int64
dtype: object

In [97]:
dataframe_restaurant['Zip']= dataframe_restaurant['Zip'].astype(str).astype(int)

In [98]:
dataframe_restaurant.dtypes

name                 object
address              object
crossStreet          object
lat                 float64
lng                 float64
labeledLatLngs       object
distance              int64
Zip                   int64
cc                   object
city                 object
state                object
country              object
formattedAddress     object
neighborhood         object
id                   object
Index                 int64
dtype: object

In [99]:
#Join on postal Code
#Merge data frames

df_restaurant = pd.merge(dataframe_restaurant, df_income, on = 'Zip')
df_restaurant.head()

,name,address,crossStreet,lat,lng,labeledLatLngs,distance,Zip,cc,city,state,country,formattedAddress,neighborhood,id,Index,Median,Mean,Pop
0,Katsu Japanese Cuisine,4th Ave,NaN,32.727264,-117.161042,"[{'label': 'display', 'lat': 32.7272640624101,...",1238,92101,US,San Diego,CA,United States,"[4th Ave, San Diego, CA 92101]",Park West,4dcc330dd22d480bd645169b,1,39102.0910,64852.7,32507
1,Sora Italian Japanese Influenced Cuisine,655 W Broadway,Kettner,32.715151,-117.168581,"[{'label': 'display', 'lat': 32.715151, 'lng':...",329,92101,US,San Diego,CA,United States,"[655 W Broadway (Kettner), San Diego, CA 92101]",Central San Diego,50651697e4b0d3feb102b8f7,1,39102.0910,64852.7,32507
2,Gyu-Kaku Japanese BBQ,801 5th Ave,F St.,32.713734,-117.159976,"[{'label': 'display', 'lat': 32.713734, 'lng':...",639,92101,US,San Diego,CA,United States,"[801 5th Ave (F St.), San Diego, CA 92101]",NaN,5f93a630aecea61d0fa246ce,1,39102.0910,64852.7,32507
3,Koi Japanese Restaurant,744 Market St,8th Ave,32.711650,-117.157730,"[{'label': 'display', 'lat': 32.71165, 'lng': ...",945,92101,US,San Diego,CA,United States,"[744 Market St (8th Ave), San Diego, CA 92101]",NaN,48701183f964a52004511fe3,1,39102.0910,64852.7,32507
4,Yoshino Japanese Restaurant,1790 W Washington St,at India St,32.743662,-117.181338,"[{'label': 'display', 'lat': 32.74366238711516...",3323,92103,US,San Diego,CA,United States,"[1790 W Washington St (at India St), San Diego...",Mission Hills,44533d88f964a520ab321fe3,2,60669.8772,85051.3,31046


The median household income of the US in 2019 was $68,703.  The zip codes for new Japanese restaurants need to have a mean income higher than this.  https://www.census.gov/library/publications/2020/demo/p60-270.html#:~:text=Median%20household%20income%20was%20%2468%2C703,and%20Table%20A%2D1).

In [110]:
df_filtered = df_restaurant[df_restaurant['Mean'] > 68703]

In [111]:
df_filtered.Zip.unique()

array([92103, 92108])

In [112]:
df_filtered.head()

,name,address,crossStreet,lat,lng,labeledLatLngs,distance,Zip,cc,city,state,country,formattedAddress,neighborhood,id,Index,Median,Mean,Pop
4,Yoshino Japanese Restaurant,1790 W Washington St,at India St,32.743662,-117.181338,"[{'label': 'display', 'lat': 32.74366238711516...",3323,92103,US,San Diego,CA,United States,"[1790 W Washington St (at India St), San Diego...",Mission Hills,44533d88f964a520ab321fe3,2,60669.8772,85051.3,31046
5,SanSai Japanese Grill,7710 Hazard Center Dr,at Frazee Rd,32.771254,-117.155877,"[{'label': 'display', 'lat': 32.77125422339901...",6125,92108,US,San Diego,CA,United States,"[7710 Hazard Center Dr (at Frazee Rd), San Die...",NaN,4ad3e184f964a520c2e620e3,3,63885.2291,74585.2,18858
6,Fuji Japanese Steakhouse & Sushi,911 Camino del Rio S,NaN,32.763673,-117.155412,"[{'label': 'display', 'lat': 32.76367258150530...",5300,92108,US,San Diego,CA,United States,"[911 Camino del Rio S, San Diego, CA 92108]",NaN,4ac82360f964a520c9bb20e3,3,63885.2291,74585.2,18858


## The filtering on US household income also excluded the zip code = 92101 which I was going to exclude anyways since it has a densly populated area of Japanese restaurants

### Lastly, will filter the results to zip codes with a minimum population of at least 20,000 inhabitants so the restaurant has enough potential customers

In [113]:
df_zip = df_filtered[df_filtered['Pop'] > 20000]

In [114]:
df_zip.Zip.unique()

array([92103])

In [132]:
#Final visualization of Zip codes where a new Japanese restaurant should go in San Diego

map = folium.Map(location = [32.743662, -117.181338], zoom_start=13)

# add the Japanese restaurants as blue circle markers
for lat, lng, in zip(df_zip.lat, df_zip.lng):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map)
map

THE END!